### Test FastALPR against our dataset

- it has different models for detection and OCR that can be tested.
- show number of overall detected plates, and put detections in JSON.
- creates a dataset with the detection for fiftyone.

In [ ]:
from fast_alpr import ALPR
import os, json
from tqdm import tqdm

detectors_to_test = ["yolo-v9-t-384-license-plate-end2end", 
                        "yolo-v9-t-512-license-plate-end2end", 
                        "yolo-v9-t-640-license-plate-end2end", 
                        "yolo-v9-s-608-license-plate-end2end"]
ocrs_to_test = ["european-plates-mobile-vit-v2-model", 
                    "global-plates-mobile-vit-v2-model"]

detectors_to_test = ["yolo-v9-t-640-license-plate-end2end"]
ocrs_to_test = ["cct-s-v1-global-model"]

def alpr_results_to_dicts(results):
    return [
        {
            "confidence": r.detection.confidence,
            "bounding_box": {
                "x1": r.detection.bounding_box.x1,
                "y1": r.detection.bounding_box.y1,
                "x2": r.detection.bounding_box.x2,
                "y2": r.detection.bounding_box.y2,
            },
            "ocr_text": r.ocr.text,
            "ocr_confidence": r.ocr.confidence
        }
        for r in results
    ]

IMAGE_DIR = "/home/gfuhr/projects/data/immediate_action/B8A44FB3A1F9__front_cars/"

First test the best detector, using the global OCR

In [ ]:
for detector in detectors_to_test:
    alpr = ALPR(
        detector_model=detector,
        ocr_model="global-plates-mobile-vit-v2-model",
    )

    filenames = [os.path.join(IMAGE_DIR, f) for f in os.listdir(IMAGE_DIR) if f.endswith('.png')]
    plates = {}

    print(f"Testing detector {detector} on {len(filenames)} images...")
    n_detected = 0
    for filename in tqdm(filenames):
        alpr_results = alpr.predict(filename)
        if alpr_results is not None and len(alpr_results) > 0:
            n_detected += len(alpr_results)
            plates[os.path.basename(filename)] = alpr_results_to_dicts(alpr_results)
        
    print(f"For detector {detector}, detected {n_detected} plates in {len(filenames)} images.")
    out_filename = f"fast_alpr_plates_{detector.replace('-license-plate-end2end', '')}.json"
    with open(out_filename, "w") as f:
        json.dump(plates, f, indent=4)

- For detector yolo-v9-t-384-license-plate-end2end, detected 159 plates in 186 images.
- For detector yolo-v9-t-512-license-plate-end2end, detected 174 plates in 186 images.
- For detector yolo-v9-t-640-license-plate-end2end, detected 189 plates in 186 images.
- For detector yolo-v9-s-608-license-plate-end2end, detected 184 plates in 186 images.

Create a fiftyone dataset to compare detections!

In [ ]:
import os
import json
from PIL import Image
import fiftyone as fo
import fiftyone.core.labels as fol

DATASET_NAME = "alpr_detection_comparison"

if DATASET_NAME in fo.list_datasets():
    fo.delete_dataset(DATASET_NAME)

dataset = fo.Dataset(name=DATASET_NAME, persistent=True)

for filename in sorted(os.listdir(IMAGE_DIR)):
    if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    img_path = os.path.join(IMAGE_DIR, filename)
    sample = fo.Sample(filepath=img_path)

    for detector in detectors_to_test:
        detections = []
        plate_data_filename = f"fast_alpr_plates_{detector.replace('-license-plate-end2end', '')}.json"
        with open(plate_data_filename, "r") as f:
            plate_data = json.load(f)

        if filename in plate_data:
            with Image.open(img_path) as img:
                img_w, img_h = img.size

            for det in plate_data[filename]:
                bbox = det["bounding_box"]
                x = bbox["x1"]
                y = bbox["y1"]
                w = bbox["x2"] - bbox["x1"]
                h = bbox["y2"] - bbox["y1"]

                # Normalize bounding box
                rel_bbox = [x / img_w, y / img_h, w / img_w, h / img_h]

                detection = fol.Detection(
                    label=detector,                    
                    bounding_box=rel_bbox,
                    confidence=det["confidence"]
                )
                #detection["ocr_confidence"] = det.get("ocr_confidence")
                detections.append(detection)

            sample[detector] = fol.Detections(detections=detections)

    dataset.add_sample(sample)
print(f"Dataset {DATASET_NAME} created with {len(dataset)} samples.")


### Now, using the apparently best detector let's compare OCRs


In [ ]:
for ocr in ocrs_to_test:

    alpr = ALPR(
        detector_model="yolo-v9-t-640-license-plate-end2end",
        ocr_model=ocr,
    )

    filenames = [os.path.join(IMAGE_DIR, f) for f in os.listdir(IMAGE_DIR) if f.endswith('.png')]
    plates = {}

    print(f"Testing OCR {ocr} on {len(filenames)} images...")
    n_detected = 0
    for filename in tqdm(filenames):
        alpr_results = alpr.predict(filename)
        if alpr_results is not None and len(alpr_results) > 0:
            n_detected += len(alpr_results)
            plates[os.path.basename(filename)] = alpr_results_to_dicts(alpr_results)
        
    print(f"For OCR {ocr}, detected {n_detected} plates in {len(filenames)} images.")
    out_filename = f"fast_alpr_{ocr.replace('-mobile-vit-v2-model', '')}.json"
    with open(out_filename, "w") as f:
        json.dump(plates, f, indent=4)

#### Create a fiftyone dataset to compare OCRs

In [ ]:


import os
import json
from PIL import Image
import fiftyone as fo
import fiftyone.core.labels as fol

DATASET_NAME = "alpr_ocr_comparison"

if DATASET_NAME in fo.list_datasets():
    fo.delete_dataset(DATASET_NAME)

ocrs_to_compare = ["global-plates-mobile-vit-v2-model", "cct-s-v1-global-model"]

dataset = fo.Dataset(name=DATASET_NAME, persistent=True)

for filename in sorted(os.listdir(IMAGE_DIR)):
    if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    img_path = os.path.join(IMAGE_DIR, filename)
    sample = fo.Sample(filepath=img_path)

    for ocr in ocrs_to_compare:
        detections = []
        plate_data_filename = f"fast_alpr_{ocr.replace('-mobile-vit-v2-model', '')}.json"
        with open(plate_data_filename, "r") as f:
            plate_data = json.load(f)

        if filename in plate_data:
            with Image.open(img_path) as img:
                img_w, img_h = img.size

            for det in plate_data[filename]:
                bbox = det["bounding_box"]
                x = bbox["x1"]
                y = bbox["y1"]
                w = bbox["x2"] - bbox["x1"]
                h = bbox["y2"] - bbox["y1"]

                # Normalize bounding box
                rel_bbox = [x / img_w, y / img_h, w / img_w, h / img_h]

                detection = fol.Detection(
                    label=det["ocr_text"],                    
                    bounding_box=rel_bbox,
                    confidence=det["ocr_confidence"]
                )
                #detection["ocr_confidence"] = det.get("ocr_confidence")
                detections.append(detection)

            sample[ocr] = fol.Detections(detections=detections)

    dataset.add_sample(sample)
print(f"Dataset {DATASET_NAME} created with {len(dataset)} samples.")
